In [17]:
import pandas as pd
import datetime
import os
import numpy as np
print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Analysis/2020_Q1
2020-05-20 14:00:28.333699


In [5]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200501-152115-660.txt",
                           dtype=str,sep="|")
df_store_list=df_store_list[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd']]
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].split(" ")[0].zfill(5))
df_store_list=df_store_list[~df_store_list['location_id'].isin(['6990','145'])]

In [12]:
df_DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2],skiprows=1)
df_DMA_zip.columns=['zip_cd','DMA']
df_DMA_zip=df_DMA_zip.drop_duplicates()

In [13]:
df_DMA_zip=df_DMA_zip.groupby("zip_cd")['DMA'].apply(list).to_frame().reset_index()

In [15]:
df_store_list=pd.merge(df_store_list,df_DMA_zip,on="zip_cd")
df_store_list=df_store_list.rename(columns={"DMA":"DMA_list"})

In [16]:
print(df_store_list['DMA_list'].apply(len).max())

3


In [18]:
df_store_list['DMA_1']=df_store_list['DMA_list'].apply(lambda x: x[0] if len(x)>=1 else np.nan )
df_store_list['DMA_2']=df_store_list['DMA_list'].apply(lambda x: x[1] if len(x)>=2 else np.nan )
df_store_list['DMA_3']=df_store_list['DMA_list'].apply(lambda x: x[2] if len(x)>=3 else np.nan )


In [20]:
writer=pd.ExcelWriter("./BL_store_list_with_DMA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_store_list.to_excel(writer,"store_list",index=False)
writer.save()